<a href="https://colab.research.google.com/github/Aswin2808/AI/blob/main/spam_or_not_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense


In [16]:
# Load the dataset in one line and select relevant columns
data_df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='latin1', usecols=[0, 1], names=['label', 'text'], header=0)

# Inspect the first few rows to ensure it is loaded correctly
print(data_df.head())


  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [18]:
# Preprocess the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data_df['text'].tolist())

sequences = tokenizer.texts_to_sequences(data_df['text'].tolist())
max_length = 100
X = pad_sequences(sequences, maxlen=max_length)

le = LabelEncoder()
y = le.fit_transform(data_df['label'])

In [19]:
# Build the GRU model
vocab_size = 10000
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model with validation split
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
140/140 [==============================] - 37s 228ms/step - loss: 0.1401 - accuracy: 0.9536 - val_loss: 0.0521 - val_accuracy: 0.9839
Epoch 2/10
140/140 [==============================] - 31s 222ms/step - loss: 0.0196 - accuracy: 0.9953 - val_loss: 0.0455 - val_accuracy: 0.9874
Epoch 3/10
140/140 [==============================] - 30s 214ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 0.0441 - val_accuracy: 0.9874
Epoch 4/10
140/140 [==============================] - 31s 221ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0550 - val_accuracy: 0.9883
Epoch 5/10
140/140 [==============================] - 30s 211ms/step - loss: 4.3660e-04 - accuracy: 1.0000 - val_loss: 0.0533 - val_accuracy: 0.9865
Epoch 6/10
140/140 [==============================] - 30s 213ms/step - loss: 1.3754e-04 - accuracy: 1.0000 - val_loss: 0.0588 - val_accuracy: 0.9883
Epoch 7/10
140/140 [==============================] - 33s 235ms/step - loss: 6.6457e-05 - accuracy: 1.0000 - val_loss: 0.0

In [23]:
# Predict the class of a new message
input_text = ["Had your contract mobile 11 Mnths? Latest Motorola, Nokia etc. all FREE! Double Mins & Text on Orange tariffs. TEXT YES for callback, no to remove from records."]
input_sequence = tokenizer.texts_to_sequences(input_text)
input_padded = pad_sequences(input_sequence, maxlen=max_length)

prediction = model.predict(input_padded)
label = "spam" if prediction >= 0.5 else "ham"
print(f'The message is: {label}')

1/1 [==============================] - 0s 59ms/step
The message is: spam
